In [11]:
import pandas as pd
import numpy as np

In [19]:
df=pd.read_excel('traduccion_pt_clean.xlsx', header=None, index_col=0)

In [18]:
df2=pd.read_excel('FOODs_Aditivos.xlsx')

In [21]:
borders=[n for n,row in df2.iterrows() if row.isna().all()] # Como cada ingrediente distinto está separado por una row de NaN's, busco las fronteras
null_row=df2.iloc[borders[0]].copy() # Genero una row vacía para insertar fronteras
reconstitutor_for_dataframe=[] # Contenedor para todas las rows modificadas
for n,node in enumerate(borders): # Se uitera sobre el index
  df_slice=df2.iloc[borders[n-1]+1:node,:] # El cachito de dataframe que representa un ingrediente.
  df_slice["con_sinonimos_agregados"]=False 
  try:
    for n, row in df.iterrows(): # itero sobre el df que contiene los nombres traducidos.
      if row.name.lower() in ' '.join(df_slice["Unnamed: 0"].dropna().str.lower()): # el join contiene todos los nombres de los sinonimos, row.name es el nombre "de referencia"
        sel=df_slice.iloc[-1,:].copy() # Copio la ultima row para modificarla y apendarla al slice luego.
        for portuguese_name in row.dropna(): # Itero sobre todos los sinonimos traducidos
          if not portuguese_name in  df_slice["Unnamed: 0"].dropna().str.lower():
            sel['Unnamed: 0']=portuguese_name # Modifico el nombre de la ultima row con el sinonimo traducido
            df_slice=df_slice.append(sel) # se agrega la row al paquete
          
            df_slice["con_sinonimos_agregados"]=True
          # break # ? 
  except AttributeError as e:
    print(row.name, n)
  df_slice=df_slice.append(null_row) # Se agrega la null row al paquete
  reconstitutor_for_dataframe.append(df_slice) # se agrega el paquete al contenedor

/tmp/ipykernel_39799/3420646523.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice["con_sinonimos_agregados"]=False


In [ ]:
pd.concat(reconstitutor_for_dataframe).reset_index(drop=True).to_excel('con_sinonimos_en_portugues.xlsx')
df_final = pd.concat(reconstitutor_for_dataframe).reset_index(drop=True)


In [ ]:
df_final.con_sinonimos_agregados

0       NaN
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
3951    0.0
3952    0.0
3953    0.0
3954    0.0
3955    NaN
Name: con_sinonimos_agregados, Length: 3956, dtype: float64